# More Earth Engine: Irrigation, Snow, and Soils Data Extraction

In this notebook, we will:
1. Use the `sample_snodas_swe` function to perform snodas snow water equivalent (SWE) extraction.
2. Use the `get_irrigation` function to perform irrigation mask extraction.
3. Use the `get_ssurgo` function to perform soils extraction.

Ensure that the Earth Engine Python API is authenticated and configured, and that the Earth Engine CLI is available in your environment.

## 1. Import Libraries and Authorize Earth Engine

In [1]:
import os
import sys
import ee

# append the project path to the environment (mine is in /home/dgketchum/PycharmProjects')
sys.path.append('/home/dgketchum/PycharmProjects/swim-rs')

from data_extraction.ee.snodas_export import sample_snodas_swe
from data_extraction.ee.ee_props import get_irrigation, get_ssurgo
from data_extraction.ee.ee_utils import is_authorized
from utils.google_bucket import list_and_copy_gcs_bucket

sys.path.insert(0, os.path.abspath('../..'))
sys.setrecursionlimit(5000)

if not is_authorized():
    ee.Authenticate()
ee.Initialize()

## 1: Extract Raster Data

Here, we're going to use functions with a similar workflow to those we used to get ETf and NDVI data from Earth Engine.

For the raster data extraction, there are three functions we need to run:

*   **`sample_snodas_swe`**:
    *   This function iterates over the daily SNODAS images in Earth Engine, extracting mean SWE for each sample plot for each day, September through May.
*   **`get_irrigation`**:
    *   This function uses IrrMapper to get statistics about the irrigation status of each plot for each year, including the fraction of the plot that was irrigated.
*   **`get_ssurgo`**:
    *   This function uses data summarized and put in a public Earth Engine asset by Charles Morton at Desert Research Institute from SSURGO (https://www.nrcs.usda.gov/resources/data-and-reports/soil-survey-geographic-database-ssurgo) to summarize plot-scale soil texture and hydraulic properties used by SWIM-RS.

In [3]:
ee_account = 'ee-dgketchum'

# Define Constants and Remote Sensing Data Paths
# TODO: remove hard-coded collecitons and use variables defined here
SWE = 'projects/earthengine-legacy/assets/projects/climate-engine/snodas/daily'


root = '/media/research/IrrigationGIS/swim'
bucket = 'wudr'
fields = 'projects/ee-dgketchum/assets/swim/mt_sid_boulder'


# We must specify which column in the shapefile represents the field's unique ID, in this case it is 'FID_1'
FEATURE_ID = 'FID_1'

### SWE Data

Notes
- Ensure that your Cloud Storage bucket has the correct permissions for Earth Engine to write to it.
- This will produce a monthly dataset for Sep - May, regardless of SWE status at the sample plots.

In [4]:
# Here, we run the sample_snodas_swe function on the uploaded asset.

chk = os.path.join(root, 'examples/tutorial/landsat/extracts/swe')
sample_snodas_swe(fields, bucket, debug=False, check_dir=None, feature_id=FEATURE_ID)


etf_inv_irr_2004
etf_inv_irr_2005
etf_inv_irr_2006
etf_inv_irr_2007
etf_inv_irr_2008
etf_inv_irr_2009
etf_inv_irr_2010
etf_inv_irr_2011
etf_inv_irr_2012
etf_inv_irr_2013
etf_inv_irr_2014
etf_inv_irr_2015
etf_inv_irr_2016
etf_inv_irr_2017
etf_inv_irr_2018
etf_inv_irr_2019
etf_inv_irr_2020
etf_inv_irr_2021
etf_inv_irr_2022
etf_inv_irr_2023
etf_irr_2004
etf_irr_2005
etf_irr_2006
etf_irr_2007
etf_irr_2008
etf_irr_2009
etf_irr_2010
etf_irr_2011
etf_irr_2012
etf_irr_2013
etf_irr_2014
etf_irr_2015
etf_irr_2016
etf_irr_2017
etf_irr_2018
etf_irr_2019
etf_irr_2020
etf_irr_2021
etf_irr_2022
etf_irr_2023


## 4: Move Data from Bucket to Local Computer

We can monitor the data extraction on the Earth Engine code editor (https://code.earthengine.google.com/) or with their task monitor (https://code.earthengine.google.com/tasks).

Once the download is complete, we can move the data from the bucket to the local directory in one of three ways:

1. Download the data manually: click 'Open in GCS' button on the task list in the code editor, then click the download button on the bucket file.
   The following require the Google Cloud SDK (https://cloud.google.com/sdk/docs/install)
3. Move the data with the command line using e.g., 'gsutil cp gs://wudr/etf_inv_irr*.csv /media/research/IrrigationGIS/swim/examples/tutorial/landsat/extracts/etf/'
4. Move the data programatically using Python's subprocess. You can use `list_and_copy_gcs_bucket`. Use 'dry_run=True' to just list what will get copied, and 'dry_run=False' to download the data:

In [4]:
command = '/home/dgketchum/google-cloud-sdk/bin/gsutil'
for mask in ['inv_irr', 'irr']:
    dst = os.path.join(root, 'examples/tutorial/landsat/extracts/etf/{}'.format(mask))
    glob_ = f'etf_{mask}'

    # list the data
    list_and_copy_gcs_bucket(command, bucket, dst, glob=glob_, dry_run=True)


gs://wudr/etf_inv_irr_2004.csv
gs://wudr/etf_inv_irr_2005.csv
gs://wudr/etf_inv_irr_2006.csv
gs://wudr/etf_inv_irr_2007.csv
gs://wudr/etf_inv_irr_2008.csv
gs://wudr/etf_inv_irr_2009.csv
gs://wudr/etf_inv_irr_2010.csv
gs://wudr/etf_inv_irr_2011.csv
gs://wudr/etf_inv_irr_2012.csv
gs://wudr/etf_inv_irr_2013.csv
gs://wudr/etf_inv_irr_2014.csv
gs://wudr/etf_inv_irr_2015.csv
gs://wudr/etf_inv_irr_2016.csv
gs://wudr/etf_inv_irr_2017.csv
gs://wudr/etf_inv_irr_2018.csv
gs://wudr/etf_inv_irr_2019.csv
gs://wudr/etf_inv_irr_2020.csv
gs://wudr/etf_inv_irr_2021.csv
gs://wudr/etf_inv_irr_2022.csv
gs://wudr/etf_inv_irr_2023.csv
gs://wudr/etf_irr_2004.csv
gs://wudr/etf_irr_2005.csv
gs://wudr/etf_irr_2006.csv
gs://wudr/etf_irr_2007.csv
gs://wudr/etf_irr_2008.csv
gs://wudr/etf_irr_2009.csv
gs://wudr/etf_irr_2010.csv
gs://wudr/etf_irr_2011.csv
gs://wudr/etf_irr_2012.csv
gs://wudr/etf_irr_2013.csv
gs://wudr/etf_irr_2014.csv
gs://wudr/etf_irr_2015.csv
gs://wudr/etf_irr_2016.csv
gs://wudr/etf_irr_2017.csv
gs

In [5]:
command = '/home/dgketchum/google-cloud-sdk/bin/gsutil'
for mask in ['inv_irr', 'irr']:
    
    dst = os.path.join(root, 'examples/tutorial/landsat/extracts/etf/{}'.format(mask))
    glob_ = f'etf_{mask}'

    # copy the data
    list_and_copy_gcs_bucket(command, bucket, dst, glob=glob_, dry_run=False)

Copying gs://wudr/etf_inv_irr_2004.csv: Copying gs://wudr/etf_inv_irr_2004.csv...
/ [1 files][ 52.6 KiB/ 52.6 KiB]                                                
Operation completed over 1 objects/52.6 KiB.                                     

Copying gs://wudr/etf_inv_irr_2005.csv: Copying gs://wudr/etf_inv_irr_2005.csv...
/ [1 files][ 50.9 KiB/ 50.9 KiB]                                                
Operation completed over 1 objects/50.9 KiB.                                     

Copying gs://wudr/etf_inv_irr_2006.csv: Copying gs://wudr/etf_inv_irr_2006.csv...
- [1 files][ 53.6 KiB/ 53.6 KiB]                                                
Operation completed over 1 objects/53.6 KiB.                                     

Copying gs://wudr/etf_inv_irr_2007.csv: Copying gs://wudr/etf_inv_irr_2007.csv...
/ [1 files][ 54.1 KiB/ 54.1 KiB]                                                
Operation completed over 1 objects/54.1 KiB.                                     

Copying gs://wud